In [0]:
WITH state_emissions AS (
  SELECT 
    state_abbr,
    SUM(CAST(REGEXP_REPLACE(`GHG emissions mtons CO2e`, '[^0-9.]', '') AS DOUBLE)) AS total_emissions
  FROM emissions.default.emissions_data
  GROUP BY state_abbr
),
all_emissions AS (
  SELECT SUM(total_emissions) AS grand_total FROM state_emissions
),
top_10_states AS (
  SELECT 
    se.state_abbr,
    se.total_emissions,
    ROUND(se.total_emissions / ae.grand_total * 100, 2) AS percent_of_total
  FROM state_emissions se
  CROSS JOIN all_emissions ae
  ORDER BY se.total_emissions DESC
  LIMIT 10
)
SELECT 
  SUM(percent_of_total) AS top_10_states_percent_of_total,
  SUM(total_emissions) AS top_10_states_total_emissions
FROM top_10_states